In [1]:
import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from scipy import spatial

import pickle

from transformers import TFAutoModel, AutoTokenizer

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage, dendrogram

from tqdm import tqdm

# Подготовка

In [2]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [3]:
# Модели BERT для токенизации
bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
df = pd.read_pickle('df_km.pickle')

In [6]:
# ищу в df не текстовые имена
for i in df.iterrows():
    text = list(i)[1][1]
#     print(list(i)[1][1])
#     text = i['name']
    if isinstance(text, str):
        pass
    else:
        print(list(i)[1][0])
        

In [8]:
# 34773 Nonetype
df[df['id'] == 34773]

,id,name,embeding,km_30,km_60,km_90,km_120,km_150,km_180,km_210,...,km_720,km_750,km_780,km_810,km_840,km_870,km_900,km_930,km_960,km_990


In [ ]:
df.drop(labels = [21422],axis = 0, inplace = True)

In [5]:
df.shape

(34891, 25)

In [6]:
df.sample(5)

,id,name,embeding,km_30,km_60,km_90,km_120,km_150,km_180,km_210,...,km_390,km_420,km_450,km_480,km_510,km_540,km_570,km_600,km_630,km_660
21245,34488,Тыквенно-апельсиновые блины с ароматом имбиря,"[-0.27180707454681396, -0.054043591022491455, ...",16,13,69,5,62,39,104,...,2,259,48,413,212,99,25,36,99,99
30781,22640,"Итальянская пицца на быстром тесте с томатами,...","[-0.41491249203681946, -0.08802352100610733, 0...",20,37,25,28,64,79,56,...,163,179,279,151,139,217,442,84,217,217
18604,30520,Чатни из перца и персиков,"[-0.32327452301979065, 0.019079290330410004, -...",25,31,54,23,38,154,182,...,270,296,292,382,407,138,71,596,617,617
21334,34617,Блинчики с сыром и зеленью,"[-0.4315938353538513, -0.10385119915008545, 0....",26,44,66,96,109,157,175,...,67,310,286,459,372,433,147,316,433,433
4191,6203,Курица с абрикосами,"[-0.3884696960449219, -0.007507386151701212, -...",29,41,54,86,76,112,96,...,224,50,275,355,410,371,438,183,371,134


In [ ]:
# piplene обработки текста и преобразование текста в эмбединги
def pipline(text):
    lem = []
    pattern = r'[^a-zA-Zа-яА-Я0-9 ]'
    text = text.lower()
    text = text.strip()
    text = re.sub(pattern, '', text)
    text = ' '.join(i for i in text.split() if i not in sw)
    for word in text.split():
        p = morph.parse(word)[0]
        lem.append(p.normal_form)
    text = ' '.join(lem)
    tok = tokenizer([text], 
               max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    text = bert(**tok)[1].numpy().tolist()[0]
    return text
    

In [ ]:
%%time
df['embeding'] = df['name'].apply(pipline)

In [ ]:
df.head()

In [ ]:
df.to_pickle('recipes.pickle')

In [ ]:
max(df['embeding'][0])

In [ ]:
df['embeding'][0]

In [ ]:
df.head()

In [7]:
list_ = list(df['embeding'])

In [ ]:
len(list_)

# cosine distance

## кисинусное растояние для поиска ближайших токенов

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
a = list([pipline('суп')])
len(a)

In [ ]:
b = list([df.iloc[0,2]])
len(b)

In [ ]:
1 - cosine_distances(a,b)

In [ ]:
dict_emb = {k:[v] for k, v in df[['id', 'embeding']].iterrows()}

In [ ]:
1 - cosine_distances(a, [list(dict_emb.values())[1][0][1]])

In [ ]:
%%time
res = []
dis = []
for k,v in dict_emb.items():
#     print(v[0][1])
    distance = cosine_distances(a, [v[0][1]])
    
    if distance > 0.6:
        dis.append(distance)
        res.append(k)

In [ ]:
len(res)

In [ ]:
for i in res:
    print(df['name'][df['id'] == i])
# res

# Метод локтя

## для определения кол-ва кластеров

In [ ]:
# determining the maximum number of clusters
# using the simple method
limit = int((df.shape[0]//2)**0.5)
 
# selecting optimal value of 'k'
# using elbow method
 
# wcss - within cluster sum of
# squared distances
wcss = {}
 
for k in range(2,limit+1):
    model = KMeans(n_clusters=k)
    model.fit(list_)
    wcss[k] = model.inertia_
     
# plotting the wcss values
# to find out the elbow value
plt.plot(wcss.keys(), wcss.values(), 'gs-')
plt.xlabel('Values of "k"')
plt.ylabel('name')
plt.show()

In [ ]:
# метод локтя 20-40 кластеров

# Kmeans

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=0).fit(list_)

In [ ]:
km_40 = kmeans.fit(list_)

In [ ]:
df['km'] = kmeans.labels_

In [ ]:
df.head()

In [8]:
# обучение на разном кол-ве кластеров для определения лучшего кол-ва
for i in tqdm(range(660, 1010, 30)):
    kmeans = KMeans(n_clusters=i, random_state=13).fit(list_)
    pickle.dump(kmeans, open(f"models/kmeans_{i}_cl.pkl", "wb"))
    df[f'km_{i}'] = kmeans.labels_
    df.to_pickle('df_km.pickle')

100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [1:22:42<00:00, 413.51s/it]


In [ ]:
df.to_pickle('df_km.pickle')

In [9]:
df.shape

(34891, 36)

# Метод Силуэта

## метод для обнаружения кол-ва кластеров( метод локтя отработал с представленными данными лучше)

In [ ]:

# determining the maximum number of clusters
# using the simple method
limit = int((df.shape[0]//2)**0.5)
 
# determining number of clusters
# using silhouette score method
for k in tqdm(range(2, limit+1)):
    model = KMeans(n_clusters=k)
    model.fit(list_)
    pred = model.predict(list_)
    score = silhouette_score(list_, pred)
    print('Silhouette Score for k = {}: {:<.3f}'.format(k, score))

# Иерархическая кластеризация

## сама определяет кол-во кластеров(данных много и они похожи между собой, метод локтя с kmeans отработали лучше)

In [ ]:
# Реализация иерархической кластеризации при помощи функции linkage
mergings = linkage(list_, method='complete')

In [ ]:
len(mergings)

In [ ]:
mergings = np.append(mergings, mergings[34889])

In [ ]:
mergings.append(mergings[34889])

In [ ]:
df['pred'] = mergings[34889]

In [ ]:
df.shape